In [2]:
# === STEP 1: Install required packages ===
# pip install pandas openpyxl scikit-learn

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

# === STEP 2: Load Labeled Training Data ===
df_train = pd.read_excel("test01.xlsx")

# Combine relevant fields into one text string
df_train["text"] = (
    df_train["Systeme"].astype(str) + " | " +
    df_train["Description"].astype(str) + " | " +
    df_train["Description de l'équipement"].astype(str)
)

# Drop rows with missing target values
df_train = df_train.dropna(subset=["Fiabilité Intégrité", "Disponibilté", "Process Safety"])

# Define features and targets
X = df_train["text"]
y = df_train[["Fiabilité Intégrité", "Disponibilté", "Process Safety"]]

# === STEP 3: Split Data for Evaluation ===
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# === STEP 4: Build and Train Model ===
model = Pipeline([
    ("tfidf", TfidfVectorizer(max_features=1000, ngram_range=(1, 2))),
    ("regressor", MultiOutputRegressor(Ridge()))
])

model.fit(X_train, y_train)

# === STEP 5: Evaluate on Test Set ===
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))
r2 = r2_score(y_test, y_pred, multioutput='raw_values')

print("📊 Model Evaluation on Test Data:")
print(f"  ➤ RMSE (Fiabilité Intégrité, Disponibilté, Process Safety): {rmse}")
print(f"  ➤ R²    (Fiabilité Intégrité, Disponibilté, Process Safety): {r2}")

# === STEP 6: Predict on New Unlabeled File ===
df_new = pd.read_excel("test01wfi.xlsx")
df_new["text"] = (
    df_new["Systeme"].astype(str) + " | " +
    df_new["Description"].astype(str) + " | " +
    df_new["Description de l'équipement"].astype(str)
)

# Predict using trained model
X_new = df_new["text"]
predictions = model.predict(X_new)

# Round and clip predictions to integer scale 1–5
predicted_scores = pd.DataFrame(predictions, columns=["Fiabilité Intégrité", "Disponibilté", "Process Safety"])
predicted_scores = predicted_scores.round().clip(1, 5).astype(int)

# Combine predictions with original data
df_result = pd.concat([df_new, predicted_scores], axis=1)

# Save to Excel
df_result.to_excel("scored_anomalies.xlsx", index=False)
print("\n✅ Scoring complete! Results saved to 'scored_anomalies.xlsx'")


📊 Model Evaluation on Test Data:
  ➤ RMSE (Fiabilité Intégrité, Disponibilté, Process Safety): [0.43052555 0.64609955 0.74934298]
  ➤ R²    (Fiabilité Intégrité, Disponibilté, Process Safety): [0.41651457 0.67834423 0.6633514 ]

✅ Scoring complete! Results saved to 'scored_anomalies.xlsx'
